In [1]:
import numpy as np

# Estimating the $L_1$ Lipschitz constant of a function

In many problems of interest in optimization and other fields, target functions $f: R^N \to R$ must have bounded slopes, in the sense that for any $x,y \in R^N$ $\exists L_1 >0$ so that

$$||\nabla f(x) - \nabla f(y)|| \leq L_1 ||x-y||,$$

where $||\cdot||$ denotes the usual Euclidean norm in $R^N$; i.e., $||x||^2=\sum_{i=1}^N x_i^2, x\in R^N$. We can call the Euclidean norm by np.linalg.norm(). Such a constant $L_1$ is called the Liptschitz constant of $f$, and we will write $L_1^*$ to denote the supremum or least upper bound of all possible values $L_1$ for which the inequality in question may hold. We need to determine $L_1^*$ to find out what the tightest bound in variations of $\nabla f(\cdot)$ are. 

In this notebook, we numerically find a value for an estimator $\hat{L_1^*}$ using approximations to derivatives in our noisy, black-box setting using a paper and results from More and Wild. Their main result provides an optimal value $h$ in a forward difference approximation to the directional derivative $f', p \in R^N$ with $$f' \approx \frac{f(x+hp)-f(x)}{h}.$$ In fact, More and Wild show that with certain assumptions, using their optimum $h$ value, this approximation is the best, in the sense of minimizing a norm in the function space $\mathcal{L}_1$; importantly the authors assume the presence of additive noise.

### Example

Let $f: R^N \to R$ equal the sphere function; i.e., $$f(x; \xi)=\sum_{i=1}^N x_i^2 + \epsilon (\xi),$$ where $\epsilon \sim U[-k,k]$ is stoachastic additive noise with zero mean and bounded variance. Then we have $\nabla f(x)=2x$ and one can show that $L_1^*=2$. This means that the value of $||\nabla f(x) -\nabla f(y)||$ will never be larger than twice the value of $2||x-y||$ which is obvious because $||\nabla f(x) -\nabla f(y)||$ literally equals $2||x-y||$.

In [96]:
k=1e-5

def sphere(x):
    xs = x**2
    return np.sum(xs, axis=0) + k*(2*np.random.rand(1) - 1)

#### Analytically getting $L_1$ (just to make sure we aren't doing anything crazy)

In [97]:
# Draw two random points in a large hypercube:

N=10

x_0=100*(2*np.random.rand(10,1) - np.ones((N,1)))
y_0=100*(2*np.random.rand(10,1) - np.ones((N,1)))

d_1=np.linalg.norm(x_0-y_0)
print(d_1)

226.38382915558242


In [98]:
# Now compute the difference in their derivatives
d_2=np.linalg.norm(2*(x_0-y_0))

print(d_2)

452.76765831116484


In [99]:
print(d_2/d_1)

2.0


#### Numerical Approach

We will compute ratios at samples $x_k, x_j \in R^N$ $$\frac{||\hat{\nabla f(x_j)}-\hat{\nabla f(x_k)}||- 2 \epsilon^*}{||x_j - x_k||},j \neq k$$  where $\hat{\nabla f (x_k)}$ denotes a numerical approximation to $\nabla f(x_k)$. Here $\epsilon^* := \sup_{\xi} \epsilon (\xi),$ a technique mentioned in a Callies paper as a slightly-better-than-ad-hoc way of estimating $L_1$: slightly better since we are trying to remove the average effect of the noise. In our example, we have $\epsilon^* \approx \bar{\epsilon} + 3\sigma$, where $\bar{\epsilon}$ is the mean of the noise and $\sigma^2$ is the variance of the noise. We estimate $\sigma$ numerically in another notebook, but here we know the mean and variance of our noise. The mean is 0 and variance is $k^2/3$. Adding three times the standard deviation to the mean is a slightly optimistic upper bound on the average contribution of noise, but should suffice.

In [100]:
std=k/np.sqrt(3)
eps_star=3*std
eps_star

1.7320508075688777e-05

Now we need estimates for the gradients at sample points, which are denoted $\hat{\nabla f (x_k)}$. We do so by evaluating the forward difference for the $i$-th component, $$\frac{f(x+ h^* \cdot e_i)-f(x)}{h^*} \quad h^*:=8^{1/4}\left(\frac{\sigma}{\mu_{f''}}\right)^{1/2} \quad \mu_{f''} \approx \max |f''| \quad e_i:=(0,\ldots,0,1,0,\ldots,0).$$ In our case, $\mu_{f''}=\sqrt{10\cdot2^2}=\sqrt{40}$ since the Hessian of $f$, $\nabla^2 f$ is a $10 \times 10$ diagonal matrix with entries of 2 along the diagonal!

In [127]:
M=2

x_vals=np.zeros((N,1)) # matrix of the vectors we randomly make

grads=np.zeros((N,1)) # matrix of the gradients we approximates

mu_f2=np.sqrt(40)
h_star=(8**0.25)*np.sqrt(std/mu_f2)

F=sphere

for j in range(0,M):

    x=100*(2*np.random.rand(N,1) - np.ones((N,1)))
    x_vals=np.hstack((x_vals,x))

    approx_grad=np.zeros((N,1))
    for i in range(0,N):
        e = np.zeros((N,1))
        e[i] = 1.0
        approx_grad[i] = (F(x + h_star*e) - F(x))/h_star
    
    grads=np.hstack((grads,approx_grad))

In [128]:
v_1=np.linalg.norm(grads[:,1]-grads[:,2]) -2*eps_star

In [129]:
v_2=np.linalg.norm(x_vals[:,1]-x_vals[:,2])

In [130]:
v_1/v_2

2.0000070058507005

### Example 2

In [133]:
# Define a noisy test function

# Noise level
k=1e-5

# Parameters in test function
A = 100 # alpha
B = 10 # beta
C = .1 # gamma

# A mix of quadratics with vertical shift of 9; can be defined from R^p -> R for any p>=6.
# The noise is a draw from U[-k,k] and is modeled in an additive fashion
def function1(a):
    x = a[0]; y = a[1]; z = a[2]; w = a[3]; p = a[4]; q = a[5]; 
    f = (A*(x+y)**2+B*(z+w)**2+C*(p+q)**2+9) # this is the true value of f
    f_pert = f + k*(2*np.random.rand(1) -1) # this is the noisy, perturbed value of f
    return f_pert[0]